In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline

## Master Template for processing each dataset
- Sentinel-1 (in order of priority)
 - **SLC via ASF using IPP Colombia**: in testing
 - **GRD via ASF using PyroSAR S1 Paper**: under development

In [3]:
import glob
import shutil
from datetime import datetime
import pandas as pd
import time
import tqdm
import os

from subprocess import Popen, PIPE, STDOUT

from utils.prepS1 import prepareS1

In [4]:
source_envars = ['AWS_ACCESS_KEY_ID','AWS_SECRET_ACCESS_KEY','AWS_S3_ENDPOINT','COPERNICUS_USERNAME','COPERNICUS_PWD',
                'S1_PROCESS_P1A','S1_PROCESS_P2A','S1_PROCESS_P3A','S1_PROCESS_P4A','SNAP_GPT','ASF_USERNAME','ASF_PWD']
for var in source_envars:
    print(var,':',os.getenv(var)!=None)

AWS_ACCESS_KEY_ID : True
AWS_SECRET_ACCESS_KEY : True
AWS_S3_ENDPOINT : False
COPERNICUS_USERNAME : True
COPERNICUS_PWD : True
S1_PROCESS_P1A : True
S1_PROCESS_P2A : True
S1_PROCESS_P3A : True
S1_PROCESS_P4A : True
SNAP_GPT : True
ASF_USERNAME : True
ASF_PWD : True


In [10]:
des_scenes_list = ['S1B_IW_SLC__1SDV_20190831T073944_20190831T074019_017826_0218C3_FFCE',
                   'S1B_IW_SLC__1SDV_20190831T073916_20190831T073946_017826_0218C3_9DD8',
                   'S1B_IW_SLC__1SDV_20190819T073943_20190819T074019_017651_021350_1E4A',
                   'S1B_IW_SLC__1SDV_20190819T073916_20190819T073946_017651_021350_2701',
                   'S1B_IW_SLC__1SDV_20190807T073943_20190807T074018_017476_020DDA_5CB6', 
                   'S1B_IW_SLC__1SDV_20190807T073915_20190807T073945_017476_020DDA_25DF', 
                   'S1B_IW_SLC__1SDV_20190726T073942_20190726T074017_017301_020896_1308']
des_scenes_list = [
    "S1A_IW_GRDH_1SDV_20190912T151117_20190912T151142_028990_0349C6_D777",
    "S1A_IW_GRDH_1SDV_20190912T151142_20190912T151207_028990_0349C6_D342",
    "S1A_IW_GRDH_1SDV_20190912T151052_20190912T151117_028990_0349C6_D2E2",
    "S1B_IW_GRDH_1SDV_20190911T151824_20190911T151849_017992_021DEF_EAF6",
    "S1B_IW_GRDH_1SDV_20190911T151849_20190911T151914_017992_021DEF_A83B"
]


test = [
    'S1A_IW_GRDH_1SDV_20191025T064008_20191025T064044_029611_035F2D_0B2A',
    'S1A_IW_GRDH_1SDV_20191013T064008_20191013T064044_029436_035929_1DCF',
    'S1A_IW_GRDH_1SDV_20191001T064008_20191001T064044_029261_035324_C74C'
]

In [5]:
err = ['S1A_IW_GRDH_1SSV_20160506T174027_20160506T174050_011141_010CE4_FBE7']

In [6]:

for s in err[:1]:
#     print(s)
    prepareS1(s, s3_dir='fiji/tom/sentinel1_testing/', inter_dir='../data/S1_ARD/')

KeyError: 'S1_PROCESS_P1'

In [ ]:
for url in urls:
    
    s3_name = url.plit('')[:]
    
    POpen("wget url down_path")
    
    s3_single_upload(down_path)
    
    
    